In [17]:
# import libraries
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn

from torch.autograd import Variable


In [23]:
data =pd.read_csv("https://bluegojifuriends.s3.us-west-1.amazonaws.com/bigT3.csv")

In [24]:
data.head()

,User_id,First_Name,Last_Name,Address,Age,Gender,Rehabilitation,Pet_intimacy,category_name,Pet_Name,log_count,com_count,avg_loading,sum_duration
0,1,Jane,Bruen,"5272 Adela Overpass Suite 982\nRozellachester,...",28,1,2,4.2,'Maltese',Barney,5,5,2611.97,70.8
1,2,Eliseo,Trantow,"2506 Mertz Inlet\nSouth Ezekielshire, OK 56215",12,1,1,1.2,'Golden Retriever',Juliana,1,2,4931.20,4.1
2,3,Oceane,Walter,"9469 Frank Orchard Apt. 685\nLake Irmastad, ID...",64,1,2,9.1,'Bulldog',Sabrina,7,3,2610.35,59.9
3,4,Zack,Marvin,"982 Susan Manors\nPort Amiya, UT 97043-2723",38,1,1,8.6,'German Shepherd',Yasmine,9,8,2727.02,212.3
4,5,Marco,Kessler,"671 Troy Isle Suite 459\nNew Carriemouth, OK 6...",28,1,1,1.9,'German Shepherd',Cristopher,4,7,4054.30,14.6


In [4]:
data.columns


Index(['User_id', 'First_Name', 'Last_Name', 'Address', 'Age', 'Gender',
       'Rehabilitation', 'Pet_intimacy', 'category_name', 'Pet_Name',
       'log_count', 'com_count', 'avg_loading', 'sum_duration'],
      dtype='object')

In [5]:
data.shape

(600, 14)

In [29]:
data.loc[:,"category_name"].unique()

array(["'Maltese'", "'Golden Retriever'", "'Bulldog'",
       "'German Shepherd'", "'Shi-ba-inu'", "'Poodle'"], dtype=object)

In [7]:
data.loc[:,"Age"].describe()

count    600.000000
mean      49.863333
std       22.283527
min       12.000000
25%       30.000000
50%       50.000000
75%       69.000000
max       90.000000
Name: Age, dtype: float64

In [8]:
data.loc[:, "Pet_intimacy"].describe()

count    600.000000
mean       4.875000
std        2.853708
min        0.000000
25%        2.500000
50%        4.850000
75%        7.300000
max       10.000000
Name: Pet_intimacy, dtype: float64

In [9]:
sum(data["avg_loading"] > data["sum_duration"])

596

In [10]:
data.loc[:, "Gender"].value_counts()

1    304
2    296
Name: Gender, dtype: int64

In [11]:
data.loc[:, "Rehabilitation"].value_counts()

2    300
1    300
Name: Rehabilitation, dtype: int64

In [14]:
def get_continent(col):
    try:
        col = col.strip()
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)


def get_country(zipcode):
    try:
        country = geolocator.geocode(zipcode, language='en').address.split(',')[-1]
        return country
    except AttributeError:
        return None

In [32]:
# write a function to address the probability of different classification
# create a user satisfication column with user satisfication level from 1 to 10
# first set weight to each variables according to experimental tests and common sense
# set up our own weight to create continuous tartget y variable using linear mapping 
# normalize the dependet variables to 0 to 1 or 0 to 10

In [18]:

def normalize_series(series):
    min_val = series.min()
    max_val = series.max()
    normalized_series = series.apply(lambda x: (x - min_val) / (max_val - min_val))
    return normalized_series

In [26]:
# Normalize every numeircal values 
data["Pet_intimacy"] = normalize_series(data["Pet_intimacy"])
data["avg_loading"] = normalize_series(data["avg_loading"])
data["sum_duration"] = normalize_series(data['sum_duration'])
data["log_count"] = normalize_series(data['log_count'])
data["com_count"] = normalize_series(data['com_count'])

Mapping the metrics in linear term

In [78]:
def calculate_user_satisfaction(row):
    # Define weights for each metric 
    # metric1: rehab, metric2, pet_intimacy, 3: category_name,4 log_count,5 com_count, 6: avg_loading_time,7 sum_duration
    weights = [0, 0.1, 0.1, 0.2, 0.1, 0.1, 0.4]
    metric1, metric2, metric3, metric4, metric5, metric6, metric7 = row[0],row[1],row[2],row[3],row[4],row[5],row[6]
    if metric1 == 1: # means it is for rehabilitation otherwise it is 0.2 
        weights[0] = 0.2
    if metric3 == "Golden Retriever": # means it is for rehabilitation otherwise it is 0.2 
        weights[2] = 0.5
    # Calculate the weighted average score
    weighted_score = (weights[0]*metric1 + weights[1]*metric2 + weights[2]*0.5 +
                      weights[3]*metric4 + weights[4]*metric5 + weights[5]*metric6+ weights[6]*metric7)

    # Return the user satisfaction score
    return weighted_score

In [70]:
new_data = data.loc[:,['Rehabilitation','Pet_intimacy','category_name',"log_count",'com_count','avg_loading',"sum_duration"]]
new_data

,Rehabilitation,Pet_intimacy,category_name,log_count,com_count,avg_loading,sum_duration
0,2,0.42,'Maltese',0.333333,0.333333,0.526948,0.268787
1,1,0.12,'Golden Retriever',0.000000,0.083333,1.000000,0.003579
2,2,0.91,'Bulldog',0.500000,0.166667,0.526618,0.225447
3,1,0.86,'German Shepherd',0.666667,0.583333,0.550415,0.831412
4,1,0.19,'German Shepherd',0.250000,0.500000,0.821139,0.045328
...,...,...,...,...,...,...,...
595,1,0.82,'Bulldog',0.250000,0.333333,0.396222,0.306561
596,2,0.91,'Shi-ba-inu',0.166667,0.500000,0.425255,0.201590
597,2,0.28,'Poodle',0.083333,0.333333,0.215269,0.004374
598,2,0.10,'Golden Retriever',0.500000,0.166667,0.427905,0.757455


In [82]:
y = new_data.apply(calculate_user_satisfaction,axis=1)
data['User_satisfication'] = y *10
new_data['User_satisfication'] = y *10

In [83]:
data

,User_id,First_Name,Last_Name,Address,Age,Gender,Rehabilitation,Pet_intimacy,category_name,Pet_Name,log_count,com_count,avg_loading,sum_duration,User_satisfication
0,1,Jane,Bruen,"5272 Adela Overpass Suite 982\nRozellachester,...",28,1,2,0.42,'Maltese',Barney,0.333333,0.333333,0.526948,0.268787,3.522098
1,2,Eliseo,Trantow,"2506 Mertz Inlet\nSouth Ezekielshire, OK 56215",12,1,1,0.12,'Golden Retriever',Juliana,0.000000,0.083333,1.000000,0.003579,3.717647
2,3,Oceane,Walter,"9469 Frank Orchard Apt. 685\nLake Irmastad, ID...",64,1,2,0.91,'Bulldog',Sabrina,0.500000,0.166667,0.526618,0.225447,4.005074
3,4,Zack,Marvin,"982 Susan Manors\nPort Amiya, UT 97043-2723",38,1,1,0.86,'German Shepherd',Yasmine,0.666667,0.583333,0.550415,0.831412,9.152728
4,5,Marco,Kessler,"671 Troy Isle Suite 459\nNew Carriemouth, OK 6...",28,1,1,0.19,'German Shepherd',Cristopher,0.250000,0.500000,0.821139,0.045328,4.692451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,Ethan,Schuppe,"17224 Koelpin Rest\nTheresiashire, GA 69914",79,2,1,0.82,'Bulldog',Filomena,0.250000,0.333333,0.396222,0.306561,5.775798
596,597,Austin,Legros,"476 Veda View\nGroverfurt, AK 15766-2282",80,1,2,0.91,'Shi-ba-inu',Herta,0.166667,0.500000,0.425255,0.201590,3.474951
597,598,Ilene,Dietrich,"934 Josianne Inlet\nLavonnechester, AZ 30057-6787",74,1,2,0.28,'Poodle',Lamont,0.083333,0.333333,0.215269,0.004374,1.512764
598,599,Novella,Becker,"76281 Germaine Valley\nBoscobury, DE 99069-3814",29,2,2,0.10,'Golden Retriever',Wendy,0.500000,0.166667,0.427905,0.757455,5.224393


New_data is the normalized data_set with user satisfication from 0 to 10
data is the original dataset with depednet varioable user satisfication from 0 to 10